# Template for measurements in elab

In [1]:
#load packages
import time
import qcodes as qc
from qcodes import Plot
from qcodes.plots.analysis_tools import IGconvert

from qcodes.instrument_drivers.QDev.QDac import QDac
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes.instrument_drivers.ZI.ZIHF2LI import ZIHF2LI
from qcodes.instrument_drivers.ZI.ZIMFLI import ZIMFLI
from qcodes.instrument_drivers.ZI.ZIMFLI_sync import syncLockins
from qcodes.instrument_drivers.yokogawa.GS200 import GS200

#---------------------------------------------------------------------------------
#Set format for saving data
#---------------------------------------------------------------------------------

qc.set_data_format('data/#{counter}_{name}_{date}_{time}')

#---------------------------------------------------------------------------------
#Connect to instruments
#---------------------------------------------------------------------------------

keith = Keithley_2600('keith','GPIB0::26::INSTR')
k1=keith.smua
k2=keith.smub

yoko=GS200('yoko','GPIB0::1::INSTR')

li_hf = ZIHF2LI(name ='li_hf', serial='dev1832')
li_a = ZIMFLI(name ='li_a', serial='dev5185', server="internal")

#------------------------------------------------------------------------
# Define custom parameters
#------------------------------------------------------------------------

voltagedivider=1e5
currentamplifier=1e-6
excitation=li_a.out0_amp()
G0=1/7.74809173e-5
cond2pt=qc.ScaledParameter(li_a.demod0_X,gain=currentamplifier*G0/(voltagedivider*excitation),name='cond2pt',label='G',unit='2e2/h')
current=qc.ScaledParameter(li_a.demod0_X,gain=currentamplifier,name='current',label='Current',unit='A')

def set_stepper(val):
    0
stepper=qc.Parameter(name='stepper',label='step',unit='',set_cmd=set_stepper)

#------------------------------------------------------------------------
# Initiate a 'station' with the instrument and parameters declared above #VERY IMPORTANT to keep updated
#------------------------------------------------------------------------

station=qc.Station(li_a,li_hf,keith,yoko,timer,stepper,cond2pt)

#------------------------------------------------------------------------
# Function Definitions
#------------------------------------------------------------------------

def device_move(devicename,end_gate,steps=100):
    start_gate = devicename()
    for i in range(0,steps+1):
        devicename(start_gate + (end_gate - start_gate)/steps * i)
        #print(start_gate + (end_gate - start_gate)/gate_points * i)
        time.sleep(0.01)

Connected to: Keithley Instruments Inc. 2612 (serial:1155500, firmware:1.4.2) in 0.16s


# Single loop

In [ ]:
station.set_measurement(li_a.demod0_X,li_a.demod0_Y)

device_name='test'
instrument_info='test'

sweep_parameter=li_a.out0_amp
start=sweep_parameter()
stop=0.1
num=101
delay=0.1

loop=qc.Loop(sweep_parameter.sweep(start,stop,num=num),delay=delay).each(*station.measure())

data=loop.get_data_set(name='{} {}({} {}){} with {}'.format(device_name,sweep_parameter.full_name,start,stop,sweep_parameter.unit,instrument_info))
data

In [ ]:
pp=qc.Plot()
data.publisher=pp
pp.add(data.li_a_demod0_X,subplot=0)
pp.add(data.timer,title='time',name='time',subplot=1)

loop.run()

# 2d loop

In [ ]:
station.set_measurement(li_a.demod0_X,li_a.demod0_Y)

device_name='test'
instrument_info='test'

sweep_parameter=li_a.out0_amp
start=0
stop=0.1
num=11
delay=0.05

step_parameter=stepper
step_start=0
step_stop=1
step_num=10
step_delay=0.1

loop=qc.Loop(sweep_parameter.sweep(start,stop,num=num),delay=delay).each(*station.measure())
loop_2d=qc.Loop(step_parameter.sweep(step_start,step_stop,num=step_num),delay=step_delay).each(loop)

data=loop_2d.get_data_set(name='{} {}({} {}){} {}({} {}){} with {}'.format(device_name,step_parameter.full_name,step_stop,step_start,step_parameter.unit,sweep_parameter.full_name,stop,start,sweep_parameter.unit,instrument_info))
data

In [ ]:
pp=qc.Plot()
data.publisher=pp
pp.add(data.li_a_demod0_X,subplot=0)
pp.add(data.timer,title='time',name='time',subplot=1,linecuts=True)
loop_2d.run()

for i in range(len(data.li_a_out0_amp_set)):
    pp.add(data.li_a_out0_amp_set[i],data.timer[i],xlabel='amplitude',xunit='V',ylabel='time',yunit='s',subplot=2)
IGexport(data,z_param='li_a_demod0_X')

# 2d loop with up and down loops

In [ ]:
station.set_measurement(timer,k1.curr,k2.curr)

device_name='device1'
instrument_info='insert here'

sweep_parameter=k1.volt
start=0
stop=1
num=11
delay=0.1

step_parameter=k2.volt
step_start=0
step_stop=1
step_num=11
step_delay=0.1

loop=qc.Loop(sweep_parameter.sweep(start,stop,num=num),delay=delay).each(*station.measure())
loop_down=qc.Loop(sweep_parameter.sweep(stop,start,num=num),delay=delay).each(*station.measure())
loop_2d_updown=qc.Loop(step_parameter.sweep(step_start,step_stop,num=step_num),delay=step_delay).each(loop,loop_down)

data=loop_2d_updown.get_data_set(name='{} {}({} {}){} {}({} {}){} with {}'.format(device_name,step_parameter.full_name,step_stop,step_start,step_parameter.unit,sweep_parameter.full_name,stop,start,sweep_parameter.unit,instrument_info))
data

In [ ]:
pp=Plot()
data.publisher=pp
pp.add(data.keith_smua_curr_0_1,title='k1 curr up',name='k1 curr up',subplot=0)
pp.add(data.keith_smub_curr_0_2,title='k2 curr up',name='k2 curr up',subplot=1)
pp.add(data.keith_smua_curr_1_1,title='k1 curr down',name='k1 curr down',subplot=2)
pp.add(data.keith_smub_curr_1_2,title='k2 curr down',name='k2 curr down',subplot=3)
_=loop_2d_updown.run()

# 3d embedded loop example

In [ ]:
pp=qc.Plot()
magneticfields=[i*0.1 for i in range(11)]
j=0
for field in magneticfields:
    magnet.field(field)
    biasloop=(k1.volt.sweep(0.1,-0.1,num=201),delay=0.1).each(*station_measure)
    outergateloop=(k2.volt.sweep(0,1,num=101),delay=2).each(biasloop).
    data=outergateloop.get_data_set('name=dev2 bias(0.1 -0.1)V gate(0 1)V with field={}T'.format(field))
    data.publisher=pp
    pp.add(data.current,title='current',name='current',subplot=j)
    outergateloop.run()
    j=j+1